In [ ]:
"""
Convert all DCM images in the folder to nii format
and preserve existing nii and ROI segmentation files
"""
import os
import shutil
import pydicom
import numpy as np
import nibabel as nib
from pathlib import Path
from collections import defaultdict
import gzip


In [ ]:
def load_dicom_series(dicom_dir):
    """Read DICOM series and convert to numpy array"""
    dicom_files = []
    
    # Collect all DCM files
    for root, dirs, files in os.walk(dicom_dir):
        for file in files:
            if file.upper().endswith('.DCM') or file.lower().endswith('.dcm'):
                dicom_path = os.path.join(root, file)
                try:
                    dicom_files.append(pydicom.dcmread(dicom_path))
                except Exception as e:
                    print(f"Warning: Unable to read {dicom_path}: {e}")
                    continue
    
    if not dicom_files:
        return None, None, None
    
    # Sort by instance number
    try:
        dicom_files.sort(key=lambda x: float(x.InstanceNumber) if hasattr(x, 'InstanceNumber') else 0)
    except:
        try:
            dicom_files.sort(key=lambda x: float(x.SliceLocation) if hasattr(x, 'SliceLocation') else 0)
        except:
            pass
    
    # Get image data
    try:
        # Get basic information from first image
        first_slice = dicom_files[0]
        rows = int(first_slice.Rows)
        cols = int(first_slice.Columns)
        num_slices = len(dicom_files)
        
        # Create 3D array
        volume = np.zeros((rows, cols, num_slices), dtype=np.float32)
        
        # Fill data
        for i, ds in enumerate(dicom_files):
            if hasattr(ds, 'pixel_array'):
                image = ds.pixel_array
                
                # Apply slope and intercept
                if hasattr(ds, 'RescaleSlope') and hasattr(ds, 'RescaleIntercept'):
                    image = image * float(ds.RescaleSlope) + float(ds.RescaleIntercept)
                
                volume[:, :, i] = image.astype(np.float32)
        
        # Get spatial information
        if hasattr(first_slice, 'PixelSpacing'):
            pixel_spacing = [float(x) for x in first_slice.PixelSpacing]
        else:
            pixel_spacing = [1.0, 1.0]
        
        if hasattr(first_slice, 'SliceThickness'):
            slice_thickness = float(first_slice.SliceThickness)
        else:
            # Calculate slice spacing
            if len(dicom_files) > 1 and hasattr(dicom_files[1], 'SliceLocation'):
                if hasattr(first_slice, 'SliceLocation'):
                    slice_thickness = abs(float(dicom_files[1].SliceLocation) - float(first_slice.SliceLocation))
                else:
                    slice_thickness = 1.0
            else:
                slice_thickness = 1.0
        
        spacing = [pixel_spacing[0], pixel_spacing[1], slice_thickness]
        
        # Get origin
        if hasattr(first_slice, 'ImagePositionPatient'):
            origin = [float(x) for x in first_slice.ImagePositionPatient]
        else:
            origin = [0.0, 0.0, 0.0]
        
        # Get orientation
        if hasattr(first_slice, 'ImageOrientationPatient'):
            orientation = [float(x) for x in first_slice.ImageOrientationPatient]
            # DICOM orientation vectors need to be converted to affine matrix
            row_cos = np.array(orientation[:3])
            col_cos = np.array(orientation[3:])
            normal = np.cross(row_cos, col_cos)
            affine = np.eye(4)
            affine[:3, 0] = row_cos * spacing[0]
            affine[:3, 1] = col_cos * spacing[1]
            affine[:3, 2] = normal * spacing[2]
            affine[:3, 3] = origin
        else:
            affine = np.eye(4)
            affine[:3, :3] = np.diag(spacing)
            affine[:3, 3] = origin
        
        return volume, affine, first_slice
    except Exception as e:
        print(f"Error: Error occurred while processing DICOM series: {e}")
        return None, None, None


In [ ]:
def convert_dcm_folder_to_nii(dcm_folder, output_path):
    """Convert folder containing DCM files to nii file"""
    volume, affine, first_slice = load_dicom_series(dcm_folder)
    
    if volume is None:
        return False
    
    try:
        # Create nibabel image object
        nii_img = nib.Nifti1Image(volume, affine)
        
        # Save as nii file
        nib.save(nii_img, output_path)
        print(f"Successfully converted: {dcm_folder} -> {output_path}")
        return True
    except Exception as e:
        print(f"Error: Failed to save nii file {output_path}: {e}")
        return False


In [ ]:
def find_dcm_folders(root_dir):
    """Find all folders containing DCM files"""
    dcm_folders = []
    
    for root, dirs, files in os.walk(root_dir):
        # Check if current folder contains DCM files
        has_dcm = False
        for file in files:
            if file.upper().endswith('.DCM') or file.lower().endswith('.dcm'):
                has_dcm = True
                break
        
        if has_dcm:
            dcm_folders.append(root)
    
    return dcm_folders


In [ ]:
def copy_nii_files(source_dir, dest_dir):
    """Copy all nii and nii.gz files"""
    nii_files = []
    
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.nii') or file.endswith('.nii.gz'):
                source_path = os.path.join(root, file)
                # Preserve relative path structure
                rel_path = os.path.relpath(root, source_dir)
                if rel_path == '.':
                    dest_subdir = dest_dir
                else:
                    dest_subdir = os.path.join(dest_dir, rel_path)
                
                os.makedirs(dest_subdir, exist_ok=True)
                dest_path = os.path.join(dest_subdir, file)
                
                try:
                    shutil.copy2(source_path, dest_path)
                    nii_files.append(dest_path)
                    print(f"Copied: {source_path} -> {dest_path}")
                except Exception as e:
                    print(f"Warning: Failed to copy file {source_path}: {e}")
    
    return nii_files


In [ ]:
def process_patient_folder(patient_folder, output_base_dir):
    """Process single patient folder"""
    patient_name = os.path.basename(patient_folder)
    output_patient_dir = os.path.join(output_base_dir, patient_name)
    os.makedirs(output_patient_dir, exist_ok=True)
    
    print(f"\nProcessing patient: {patient_name}")
    print(f"Output directory: {output_patient_dir}")
    
    # 1. Find all folders containing DCM files
    dcm_folders = find_dcm_folders(patient_folder)
    
    # 2. Create corresponding output folders for each DCM folder and convert
    converted_count = 0
    for dcm_folder in dcm_folders:
        # Get relative path
        rel_path = os.path.relpath(dcm_folder, patient_folder)
        
        # Create output folder (use folder name as nii filename)
        folder_name = os.path.basename(dcm_folder)
        # Clean filename, remove disallowed characters
        safe_name = "".join(c for c in folder_name if c.isalnum() or c in ('_', '-', '.'))[:100]
        
        output_folder = os.path.join(output_patient_dir, rel_path)
        os.makedirs(output_folder, exist_ok=True)
        
        # Generate output nii filename
        output_nii = os.path.join(output_folder, f"{safe_name}.nii")
        
        # Convert DCM to nii
        if convert_dcm_folder_to_nii(dcm_folder, output_nii):
            converted_count += 1
    
    print(f"Converted {converted_count} DCM folders")
    
    # 3. Copy all existing nii and nii.gz files (including ROI segmentation)
    copied_files = copy_nii_files(patient_folder, output_patient_dir)
    print(f"Copied {len(copied_files)} nii files")
    
    return converted_count, len(copied_files)


In [ ]:
def main():
    # Set paths
    source_dir = "鼓楼-复发难治hl"
    output_dir = "鼓楼-复发难治hl_nii"
    
    if not os.path.exists(source_dir):
        print(f"Error: Source directory does not exist: {source_dir}")
        return
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"Starting processing...")
    print(f"Source directory: {source_dir}")
    print(f"Output directory: {output_dir}")
    
    # Get all patient folders
    patient_folders = []
    for item in os.listdir(source_dir):
        item_path = os.path.join(source_dir, item)
        if os.path.isdir(item_path):
            patient_folders.append(item_path)
    
    print(f"Found {len(patient_folders)} patient folders")
    
    # Process each patient
    total_converted = 0
    total_copied = 0
    
    for i, patient_folder in enumerate(patient_folders, 1):
        print(f"\n{'='*60}")
        print(f"Progress: {i}/{len(patient_folders)}")
        
        try:
            converted, copied = process_patient_folder(patient_folder, output_dir)
            total_converted += converted
            total_copied += copied
        except Exception as e:
            print(f"Error: Error occurred while processing patient folder {patient_folder}: {e}")
            continue
    
    print(f"\n{'='*60}")
    print(f"Processing completed!")
    print(f"Total converted {total_converted} DCM folders")
    print(f"Total copied {total_copied} nii files")
    print(f"Output directory: {output_dir}")


In [ ]:
# Execute main function
if __name__ == "__main__":
    main()
